In [4]:
!pip install langchain langchain-core langchain-community chromadb langchain-openai pypdf langchain_chroma chromadb --quiet

In [5]:
from pypdf import PdfReader

reader = PdfReader("../resources/books/book1.pdf")
book1 = ""
book2 = ""

for page in reader.pages:
    book1 += page.extract_text() 

reader = PdfReader("../resources/books/book2.pdf")

for page in reader.pages:
    book2 += page.extract_text() 
    
import re
book1 = re.sub(r"\s{2,}", " ", book1)
book2 = re.sub(r"\s{2,}", " ", book2)

In [6]:
from langchain_core.documents import Document
document_book1   = [Document(page_content = book1,   metadata = {"key": "value1"})]
document_book2   = [Document(page_content = book2,   metadata = {"key": "value2"})]

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 200,
    length_function = len,
)
text_splitter_bigger_docs = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len,
)
chunks_book1 = text_splitter.split_documents(document_book1)
chunks_book2 = text_splitter_bigger_docs.split_documents(document_book2)

In [8]:
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [15]:
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Connect to the container
client = chromadb.HttpClient(host='chroma', port=8000, settings=chromadb.Settings(allow_reset=True, anonymized_telemetry=False))

print(client.heartbeat())
# Connect to the SPECIFIC existing collection
db = Chroma(
    client=client,
    collection_name="kb1",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
)

1768962798233402313


In [ ]:
db.get(where={"key":"value2"})

In [24]:
query = "o que tomar no café da manha"

docs = db.similarity_search_with_relevance_scores(query, k=5, filter={"key":"value1"})

for i, d in enumerate(docs, 1):
    print(f"\n--- Resultado {i} ---")
    print("Metadata:", d[0].metadata)
    print("Texto:", d[0].page_content[:400])



--- Resultado 1 ---
Metadata: {'key': 'value1'}
Texto: da minha vida: um ovo, uma fatia de pão, uma xícara de chá. Não me esqueceu esta circunstância mí nima; no meio de tanta coisa importante obliterada escapou esse almoço. A razão principal poderia ser justamente o meu desastre; ma s não foi; a principal razão foi a reflexão que me fez o Quincas Borba, cuja visita recebi naquele dia. Disse-me ele que a frugalidade não era necessária para entender o 

--- Resultado 2 ---
Metadata: {'key': 'value1'}
Texto: isso dava uma combinação assaz complexa e vaga, uma coisa que não podereis entender, como eu não entendi. Suponhamos que não disse nada. CAPÍTULO CIX / O FILÓSOFO Sabido que reli a carta, antes e de pois do almoço, sabido fica que almocei, e só resta dizer que essa refeição foi das mais parcas da minha vida: um ovo, uma fatia de pão, uma xícara de chá. Não me esqueceu esta circunstância mí nima; n

--- Resultado 3 ---
Metadata: {'key': 'value1'}
Texto: ni-ti-va! repetiu, batendo as 

In [13]:
d1 = docs.pop()

In [23]:
d1[1]


0.6930454927460987